<a href="https://colab.research.google.com/github/realmeelo/AI_Assignment/blob/main/Capstone_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Libraries
!pip install tensorflow opencv-python scikit-learn tf2onnx onnx pillow

In [2]:
#@title Import Libraries
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import tf2onnx
import onnx


In [3]:
#@title Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("="*60)
print("FALL ARMYWORM DETECTION - AI MODEL TRAINING")
print("="*60)


FALL ARMYWORM DETECTION - AI MODEL TRAINING


In [4]:
#@title Clone Dataset Repository
!git clone https://github.com/obasekore/Spodopera_DL_dataset.git /content/Spodopera_DL_dataset

fatal: destination path '/content/Spodopera_DL_dataset' already exists and is not an empty directory.


In [21]:
#@title Configuration
DATA_PATH = '/content/Spodopera_DL_dataset'  #Change this to your dataset path
IMG_SIZE = (224, 224)
NUM_CLASSES = 3  #Adjust based on your classes
EPOCHS = 20
BATCH_SIZE = 32
MODEL_TYPE = 'transfer_learning'  #'cnn' or 'transfer_learning'

In [22]:
#@title Load Dataset
print("\n[1/7] Loading dataset...")
images = []
labels = []

print(f"Checking DATA_PATH: {DATA_PATH}")
print(f"Does DATA_PATH exist? {os.path.exists(DATA_PATH)}")

# Get class names by listing subdirectories in the data path
class_names = sorted([d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))])

for class_idx, class_name in enumerate(class_names):
    class_path = os.path.join(DATA_PATH, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"Loading {class_name}...")
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        if os.path.isfile(img_path):  # Added check to process only files
            try:
                from PIL import Image
                img = Image.open(img_path).convert('RGB')
                img = img.resize(IMG_SIZE)
                images.append(np.array(img)) # Convert PIL image to numpy array
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

X = np.array(images)
y = np.array(labels)
print(f"✓ Loaded {len(X)} images across {len(class_names)} classes: {class_names}")


[1/7] Loading dataset...
Checking DATA_PATH: /content/Spodopera_DL_dataset
Does DATA_PATH exist? True
Loading .git...
Error loading /content/Spodopera_DL_dataset/.git/HEAD: cannot identify image file '/content/Spodopera_DL_dataset/.git/HEAD'
Error loading /content/Spodopera_DL_dataset/.git/description: cannot identify image file '/content/Spodopera_DL_dataset/.git/description'
Error loading /content/Spodopera_DL_dataset/.git/config: cannot identify image file '/content/Spodopera_DL_dataset/.git/config'
Error loading /content/Spodopera_DL_dataset/.git/packed-refs: cannot identify image file '/content/Spodopera_DL_dataset/.git/packed-refs'
Error loading /content/Spodopera_DL_dataset/.git/index: cannot identify image file '/content/Spodopera_DL_dataset/.git/index'
Loading all...
Error loading /content/Spodopera_DL_dataset/all/WhatsApp Image 2018-10-26 at 10.53.20 AM.xml: cannot identify image file '/content/Spodopera_DL_dataset/all/WhatsApp Image 2018-10-26 at 10.53.20 AM.xml'
Error load

In [26]:
#@title Preprocess Data
print("\n[2/7] Preprocessing data...")


[2/7] Preprocessing data...


In [24]:
#@title Split Data
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp
)


In [27]:
#@title Normalize Pixel Values
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(f"✓ Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")


✓ Train: (733, 224, 224, 3), Val: (105, 224, 224, 3), Test: (210, 224, 224, 3)


In [28]:
#@title Build Model
print(f"\n[3/7] Building {MODEL_TYPE} model...")

if MODEL_TYPE == 'transfer_learning':
    # Transfer Learning Model (MobileNetV2)
    base_model = keras.applications.MobileNetV2(
        input_shape=(*IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
else:
    # Custom CNN Model
    model = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),

        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(*IMG_SIZE, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

model.summary()
print("✓ Model built successfully")




[3/7] Building transfer_learning model...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip (RandomFlip)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ ?                      │   0 (unbuilt) │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

✓ Model built successfully


In [29]:
#@title Compile & Train Model
print(f"\n[4/7] Training model for {EPOCHS} epochs...")

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7
)

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("✓ Training complete")




[4/7] Training model for 20 epochs...
Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.8514 - loss: 0.3738 - val_accuracy: 0.9810 - val_loss: 0.0422 - learning_rate: 0.0010
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.9843 - loss: 0.0368 - val_accuracy: 0.9905 - val_loss: 0.0170 - learning_rate: 0.0010
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9779 - loss: 0.0417 - val_accuracy: 0.9905 - val_loss: 0.0175 - learning_rate: 0.0010
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9930 - loss: 0.0157 - val_accuracy: 0.9905 - val_loss: 0.0149 - learning_rate: 0.0010
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.9902 - loss: 0.0395 - val_accuracy: 0.9905 - val_loss: 0.0165 - learning_rate: 0.0010
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.9972 - loss: 0.0118 - val_accuracy: 0.9905 - val_loss: 0.0114 - learning_rate: 0.0010
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: